In [1]:
import pickle
with open('./hw1_data/data.pickle','rb') as f:
    data = pickle.load(f)
#print data.keys()
#[u'database_name', u'query_identity', u'database_identity', u'query_name', u'database_feature', u'query_feature']
src = data['database_feature']


In [2]:
# read attributes
with open('./hw1_data/lfw_attributes.txt') as f:
    attributes = f.readlines()


In [3]:
# l2_distance.py
import numpy as np
# from sklearn.metrics import average_precision_score

# return index list with decreasing similarity
def l2_distance(database, query):
    q = query.ravel()
    
    #check feature dimension
    assert len(q) == 4720
    assert len(database.shape) == 2 and database.shape[1] == 4720
    distances = []
    for x in database:
        distances.append(np.linalg.norm(x-q,ord=2))
    return np.argsort(distances)


# calculate_map.py
def ap(idx_list, y):
    hit = 0
    n = 0
    precisions = []
    true_label_count = sum(y)
    for i in idx_list:
        y_true = y[i]
        n +=1
        if y_true == 1:
            hit += 1
            
            precisions.append(float(hit)/n)
            if hit == true_label_count:
                break
    ap = np.sum(precisions) / true_label_count
    return ap

def calculate_map(database, labels, queries, query_labels, similarity_func):
    APs = [] 
    i = 1
    for query,label in zip(queries,query_labels):
        idx_list = similarity_func(database, query)
        y = [1 if l == label else 0 for l in labels]
        APs.append(ap(idx_list, y))
#         print i
        i+=1
    mAP = np.mean(APs)
    print 'mAP : ', mAP 
    
database = data['database_feature']
labels = data['database_name'].ravel()

queries = data['query_feature']
query_labels = data['query_name'].ravel()
# calculate_map(database, labels, queries, query_labels, l2_distance)
# print'mAP :  0.10249282626788106'


In [4]:
import spams
import numpy as np
print src.shape
num = src.shape[0]
X = src.reshape([num, 80,59])

X = np.swapaxes(X,axis1=0,axis2=2)

# X = src.reshape([num, 59,80])
# X = np.swapaxes(X,axis1=0,axis2=1)
# X = np.swapaxes(X,axis1=1,axis2=2)
lambda1 = [10**i for i in range(-6,-3,1)]
K = range(100,3200,400)
param = { 'K' : 400, # learns a dictionary with 100 elements
          'lambda1' : 10**-2, 'numThreads' : -1, 'mode':2,
          'iter' : 1000}

    
print X.shape




(13113, 4720)
(59, 80, 13113)


In [5]:
import scipy
import scipy.spatial
import numpy as np
import spams
# print D
pathces_num = 80
test = data['query_feature']
num = test.shape[0]
qs = test.reshape([num, 80,59])
qs = np.swapaxes(qs,axis1=0,axis2=2)
# 
with open('./dict.npy','rb') as f:
    D_list = np.load(f)
    D_list = [np.asfortranarray(D) for D in D_list]
def get_codeword(lambda1,D_list, X):
    
    param = { 'lambda1' : lambda1, 'lambda2' : 0,'numThreads' : -1, 'mode':2}
    buf = []
    for i in range(pathces_num):
        q = np.asfortranarray(X[:,i,:])
        D = D_list[i]
        a = spams.lasso(q,D=D,return_reg_path = False,**param)
        buf.append(a)
    buf = scipy.sparse.vstack(buf)
    codeword = np.zeros(buf.shape, dtype=np.bool)
    codeword[buf.nonzero()] = True
    return codeword

#
print src.shape
num = src.shape[0]
X = src.reshape([num, 80,59])

X = np.swapaxes(X,axis1=0,axis2=2)
print X.shape
# 
q_codeword = get_codeword(10**-2, D_list, qs)
codeword = get_codeword(10**-2, D_list, X)
print q_codeword.shape
print codeword.shape


(13113, 4720)
(59, 80, 13113)
(32000, 120)
(32000, 13113)


In [7]:
import numpy as np
# from sklearn.metrics import average_precision_score

# 注意要計算and而不是xor(hamming distance)
def similarity_sparse_coding(codeword, query):
    q = query.ravel()
    
    distances = []
    for x in codeword:
#         distances.append(scipy.spatial.distance.hamming(x,query))
        distances.append(np.sum(np.logical_and(x,q)))
    return reversed(np.argsort(distances))


database = np.swapaxes(codeword, axis1=0, axis2=1)
labels = data['database_name'].ravel()

queries = np.swapaxes(q_codeword, axis1=0, axis2=1)
query_labels = data['query_name'].ravel()
print database.shape, queries.shape


import time
t1 = time.time()
calculate_map(database, labels, queries, query_labels, similarity_sparse_coding)
# mAP :  0.13721717999431532
print 'cost time : ', time.time() - t1

NameError: name 'codeword' is not defined

In [6]:
# inverted index for ranking
import threading
import numpy as np
# input : database[doc_num, word_dim]
# output : inverted_list[word_dim] each entry contains a list for docs index.
def generate_inverted_list(database, thread_count = 16):
    assert len(database.shape) == 2
    num, dim = database.shape
    print 'generating inverted list with %d doc, %d interested words.' % (num, dim)
    inv_list = [[] for i in range(dim)]
    
    def count_occurrence(database, dim, interval, inv_list, semaphore):
        X = database[:,dim: dim+interval]
        for idx in range(len(database)):
            for i in range(interval):
                if X[idx,i] == True:
                    inv_list[dim+1].append(idx)
        semaphore.release()
    
    semaphore = threading.Semaphore(value=thread_count)
    
    interval = 1000
    for d in range(0,dim, interval):
        semaphore.acquire()
        t = threading.Thread(target=count_occurrence, args=(database, d, interval, inv_list, semaphore), name='count_dim%d'%d )
        t.start()
    
#     for idx,x in enumerate(database):
#         for i,j in enumerate(x):
#             if j == True:
#                 inverted_list[i].append(idx)
    return inv_list
#output : return an idx_list with decresing similarity which depends on occurrence count in inverted list
database = np.swapaxes(codeword, axis1=0, axis2=1)
# database = database[:3000,:]
print database.shape
import time
t1 = time.time()
inv_list = generate_inverted_list(database)
t2 = time.time()
print 'takes : ', t2-t1

def generate_inverted_list2(database):
    assert len(database.shape) == 2
    num, dim = database.shape
    print 'generating inverted list with %d doc, %d interested words.' % (num, dim)
    inv_list = [[] for i in range(dim)]
    
    for idx,x in enumerate(database):
        for i,j in enumerate(x):
            if j == True:
                inv_list[i].append(idx)
    return inv_list
inv_list2 = generate_inverted_list2(database)
t3 = time.time()
print 'takes : ', t3-t2
    

(13113, 32000)
generating inverted list with 13113 doc, 32000 interested words.
takes :  326.134527206
generating inverted list with 13113 doc, 32000 interested words.
takes :  375.934621811


In [77]:
import pickle
q = queries[0,:]

def rank_by_inverted_list(database, inverted_list, query):
    assert len(query.shape) == 1
    assert len(database.shape) == 2
    num, _ = database.shape
    count = [0] * num
    for i,j in enumerate(query):
        if j == True:
            for idx in inverted_list[i]:
                count[idx] += 1
    return reversed(np.argsort(count))
# print inv_list
database = np.swapaxes(codeword, axis1=0, axis2=1)
labels = data['database_name'].ravel()

queries = np.swapaxes(q_codeword, axis1=0, axis2=1)
query_labels = data['query_name'].ravel()
print database.shape, queries.shape

def similarity_with_inverted_list(codeword, query):
    return rank_by_inverted_list(codeword, inv_list, query)


# calculate_map(database, labels, queries, query_labels, similarity_with_inverted_list)
with open('inv_list.pickle','wb') as f:
    pickle.dump(inv_list, f, protocol=2)
print 'Done'


(13113, 32000) (120, 32000)
Done


In [2]:
import pickle
with open('inv_list.pickle','rb') as f:
    inv_list = pickle.load(f)
print type(inv_list)

<type 'list'>


In [ ]:
# Pickle transform from protocol 3 to 2
# Must be kernel python3
import pickle
def trans(src, dst):
    with open(src,'rb') as f:
        data = pickle.load(f)
    with open(dst,'wb') as f:
        pickle.dump(data, f, protocol=2)
    